In [ ]:
import zipfile
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as zip_ref:
    zip_ref.extractall('train')
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as zip_ref:
    zip_ref.extractall('test')


In [ ]:
import os
import shutil

# Получаем список файлов в папке 'train'
train_filenames = os.listdir('train/train')

# Фильтруем файлы по категориям (коты и собаки)
train_cat = [filename for filename in train_filenames if filename.startswith('cat')]
train_dog = [filename for filename in train_filenames if filename.startswith('dog')]

# Создаем папку 'train/cat', если она не существует
os.makedirs('train/cat', exist_ok=True)

# Создаем папку 'train/dog', если она не существует
os.makedirs('train/dog', exist_ok=True)

# Перемещаем изображения из папки 'train' в 'train/cat' (коты)
for filename in train_cat:
    src_path = os.path.join('train/train', filename)
    dest_path = os.path.join('train/cat', filename)
    shutil.move(src_path, dest_path)

# Перемещаем изображения из папки 'train' в 'train/dog' (собаки)
for filename in train_dog:
    src_path = os.path.join('train/train', filename)
    dest_path = os.path.join('train/dog', filename)
    shutil.move(src_path, dest_path)
    
# Удаляем лишнюю папку
!cd train && rm -r train

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import *
import h5py

# Определите, сколько GPU доступно в системе
num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    # Создайте стратегию для использования нескольких GPU
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:{}".format(i) for i in range(num_gpus)])

    # Создайте модель внутри стратегии
    with strategy.scope():
        base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
        model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)

    train = model.predict(train_generator, steps=train_generator.n)
    test = model.predict(test_generator, steps=test_generator.n)
    
    # Создайте файл HDF5 и сохраните модель внутри него
    with h5py.File("gap_%s.h5" % MODEL.__name__, 'w') as h:
        # Сохраните модель
        model.save(h, "model")
        
        # Сохраните данные
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)



In [ ]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [ ]:
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=50, validation_split=0.2)

In [ ]:
model.save('model.h5')

In [ ]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

In [ ]:
import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv")

image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.at[index-1, 'label'] = y_pred[i]

dog_threshold = 0.95
cat_threshold = 0.05

# Функция для определения класса
def determine_class(label):
    if label >= dog_threshold:
        return "это пес"
    elif label <= cat_threshold:
        return "это кит"
    else:
        return "хз"

df['class'] = df['label'].apply(determine_class)
df.to_csv('pred.csv', index=None)
df.head(10)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import pandas as pd
import io
# Для загрузки изображения
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
display(upload_button)

# Для отображения загруженного изображения
output_image = widgets.Output()
display(output_image)

# Переменная для хранения загруженного изображения
img = None

# Обработчик нажатия кнопки
def handle_upload(change):
    uploaded_filename = next(iter(upload_button.value))
    content = upload_button.value[uploaded_filename]['content']
    global img
    
    with output_image:
        output_image.clear_output()
        img = Image.open(io.BytesIO(content))
        display(img)
    
    # Загружаем данные из файла pred.csv
    pred_df = pd.read_csv('pred.csv')
    
    # Извлекаем данные для данного изображения (здесь предполагается, что изображение имеет имя 'uploaded_image.jpg')
    image_filename = int(uploaded_filename.split('.')[0])
    image_data = pred_df[pred_df['id'] == image_filename]
    
    # Отображаем данные
    with output_image:
        display(image_data)

# Привязываем обработчик к кнопке
upload_button.observe(handle_upload, names='value')



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Загрузка DataFrame с предсказаниями
df = pd.read_csv("pred.csv")

# Подсчет количества записей со значением 'хз' в столбце "class"
unknown_count = (df['class'] == 'хз').sum()

# Подсчет общего количества записей
total_count = len(df)

# Вычисление доли класса 'хз'
unknown_fraction = unknown_count / total_count

# Создание графика
plt.figure(figsize=(8, 6))
plt.bar(['хз', 'Другие классы'], [unknown_count, total_count - unknown_count], color=['red', 'blue'])
plt.xlabel('Класс')
plt.ylabel('Количество записей')
plt.title('Соотношение класса "хз" к другим классам')
plt.text(0, unknown_count + 100, f'Доля "хз": {unknown_fraction:.2%}', ha='center')
plt.show()


In [ ]:


# Создание отдельного DataFrame с классом 'хз'
unknown_df = df[df['class'] == 'хз']

# Построение графика распределения столбца 'label'
plt.figure(figsize=(8, 6))
plt.hist(unknown_df['label'], bins=20, color='red', alpha=0.7)
plt.xlabel('Значение Label')
plt.ylabel('Количество записей')
plt.title('Распределение столбца Label для класса "хз"')
plt.show()


In [ ]:
!pip install ipywidgets


In [ ]:
img